In [1]:
import pandas as pd
import tensorflow as tf
import pyodbc
import os
import os.path
from datetime import date
from datetime import datetime
from datetime import timedelta
import openpyxl.utils.datetime
from openpyxl import workbook, load_workbook
import xlsxwriter
import logging
import traceback
from traceback import format_exc
from smtplib import SMTP
from openpyxl.chart import BarChart,Reference

In [2]:
def send_email(filename, recipients, body, subject):
    emailfrom = "ReportSender@wmmhc.org"
    subject = 'No Show Machine Learning Model'
    body = 'No Show Machine Learning Model |' \
           '***This report was generated automatically*** \
\n\n\n\  If there are any issues \
or if anything appears to be an error please contact ReportSender@wmmhc.org or wade.r.bouley@wmmhc.org.\nThank you!'

    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = ",".join(recipients)
    msg["Subject"] = subject
    msg.attach(MIMEText(body, 'plain'))

    filename = excel_file
    attachment = open(str(filename), 'rb')

    part = MIMEBase('application', "octet-stream")
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", "attachment; filename= " + filename)

    msg.attach(part)
    msg.as_string()
    server = SMTP("d172237.o.ess.barracudanetworks.com")
    server.sendmail(emailfrom, recipients, msg.as_string())
    server.quit()


cursor = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                        'SERVER=server_name;'
                        'DATABASE=database_name;'
                        'UID=user_name;'
                        'PWD=password')


query = '''Machine_learning_query'''

In [3]:
df = pd.read_sql(query, cursor)

C:\Users\wbouley\AppData\Local\Temp\ipykernel_19532\2131475659.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cursor)


In [5]:
df.drop('event_log_id', axis=1, inplace=True)

In [6]:
pd.options.display.max_columns = None

# Cleaning the data

In [8]:
df['reason_for_no_show'].unique()

# df.replace({'male': 1, 'female': 0}, inplace=True)
# df.replace({'Survived': 1, 'Dead': 0}, inplace=True)

array([None, 'Client No Show - No Call', 'Client Cancelled',
       'Client Cancelled - Called', 'Therapist Cancelled'], dtype=object)

In [9]:
reasons_dict = {'None': 0, 'Client No Show - No Call': 1, 'Client Cancelled': 2,
                'Client Cancelled - Called': 3, 'Therapist Cancelled': 4}
no_show_dict = {'False': 0, 'True': 1}
veteran_dict = {'False': 0, 'True': 1}
gender_dict = {'Male': 0, 'Female': 1, 'Transgender - presents as Male': 2, 'Transgender - presents as Female': 3, 'Unknown': 4}

In [10]:
df.gender.unique()

array(['Female', 'Male', 'Transgender - presents as Male',
       'Transgender - presents as Female', 'Unknown'], dtype=object)

In [11]:
df['reason_for_no_show'].replace(reasons_dict, inplace=True)
df['reason_for_no_show'].fillna(0, inplace=True)
df['is_noshow'].replace(no_show_dict, inplace=True)
df['is_veteran'].replace(veteran_dict, inplace=True)
df['gender'].replace(gender_dict, inplace=True)

### changing data types

In [12]:
df['reason_for_no_show'] = df['reason_for_no_show'].astype(int)
df['is_veteran'] = df['is_veteran'].astype(int)
df['is_noshow'] = df['is_noshow'].astype(int)

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
from sklearn.metrics import accuracy_score, confusion_matrix


In [15]:
df.corr().abs()[["is_noshow"]]

C:\Users\wbouley\AppData\Local\Temp\ipykernel_19532\4245791344.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr().abs()[["is_noshow"]]


,is_noshow
gender,0.017068
is_veteran,0.004693
is_noshow,1.000000
reason_for_no_show,0.538898


In [16]:
ml_df = df[['id_no', 'gender', 'is_veteran', 'event_name', 'profile_name', 'program_name', 'is_noshow', 'reason_for_no_show']]

In [17]:
print(ml_df['event_name'].nunique())
print(ml_df['program_name'].nunique())
print(ml_df['profile_name'].nunique())
print(ml_df['id_no'].nunique())

102
25
171
13248


In [18]:
# event_one_hot = enc.fit_transform(ml_df[['event_name']])
# program_one_hot = enc.fit_transform(df[['program_name']])
# profile_one_hot = enc.fit_transform(df[['profile_name']])

In [19]:
# event_one_hot = pd.DataFrame(event_one_hot.toarray(), columns=enc.get_feature_names_out(['event_name']))
# program_one_hot = pd.DataFrame(program_one_hot.toarray(), columns=enc.get_feature_names_out(['program_name']))
# profile_one_hot = pd.DataFrame(profile_one_hot.toarray(), columns=enc.get_feature_names_out(['profile_name']))

# ml_df2 = pd.merge(ml_df, df, on='id_no', how='inner')

In [38]:
ml_df = pd.get_dummies(ml_df, columns=['event_name', 'profile_name', 'program_name'])


In [39]:
ml_df.head()

,id_no,gender,is_veteran,is_noshow,reason_for_no_show,event_name_2nd Eval / Court Testimony,event_name_72 Hour Crisis Stabilization Form,event_name_A1C,event_name_ACT Assessment,event_name_ACT No Show Fee - Class,event_name_ACT Prime for Life Class,event_name_ADT Group Note,event_name_AIMS,event_name_AMDD - AMDD Correspondence,event_name_Admin Note (Non-Billable),event_name_Adult Day Treatment,event_name_Brief Med Note,event_name_CASII,event_name_CBPR&S Group Support,event_name_CBPR&S Individual Support,event_name_CM Assessment,event_name_CM Linkage,event_name_CM Outreach/Advocacy,event_name_CM Planning,event_name_CM Support/Monitoring,event_name_CSCT Behavioral Management,event_name_CSCT Quarterly Summary,event_name_CSCT Services,event_name_Care Coordination,event_name_Case Consultation,event_name_Clinical Assessment Approval,event_name_Coaching/ Education for Family/Advocates of SD Ind,event_name_Consultative Care,event_name_Crisis Daily Note,event_name_Crisis Facility Discharge Plan,event_name_Crisis Intervention and Response Form,event_name_Crisis Intervention and Response Program,event_name_Crisis Treatment Plan,event_name_DBT Group Skills Development,event_name_DBT Individual Skills Development,event_name_DLA20,event_name_Diagnostic Evaluation,event_name_Diagnostic Evaluation w/ Med,event_name_Discharge Summary,event_name_EM Service,event_name_Emergency Detention Discharge Plan,event_name_Emergency Phone,event_name_Emergency Phone - Collateral,event_name_Emergency Services Consultation Summary,event_name_Family Psychotherapy w/ Member Present,event_name_Family Psychotherapy w/o Member Present,event_name_Group Behavioral Management,event_name_Group Home Note,event_name_Group Psychotherapy,event_name_HIV/Hepatitis Screening,event_name_Home Support Services,event_name_IMR Group Support,event_name_IMR Individual Support,event_name_Individual Psychotherapy,event_name_Individual Psychotherapy Discharge,event_name_Individual Psychotherapy with EM,event_name_Inpatient Discharge,event_name_Lipid Panel,event_name_Med Prompt,event_name_Monthly Progress Note,event_name_Multi Family Psychotherapy,event_name_Nursing Note,event_name_Nursing Note - 99211,event_name_Nursing- - Nursing Note - 99211,event_name_PACT Daily Log,event_name_PACT Daily Rate,event_name_PACT Monthly Summary,event_name_PATH Agency Contact,event_name_PATH Individual Contact,event_name_Peer Support,event_name_Post Discharge Note - Non-billable,event_name_Psychotherapy - groups of clients,event_name_Psychotherapy for Crisis,event_name_Psychotherapy for Crisis - Old,event_name_RCM Discharge Recommendations,event_name_Rehab Aide,event_name_Risk Assessment,event_name_SDMI and LOI Worksheet,event_name_SED Assessment,event_name_SUD Assessment,event_name_SUD Assessment - Grant Based,event_name_SUD Assessment - Grant Based - V2,event_name_SUD Assessment V2,event_name_SUD Brief Assessment,event_name_SUD Brief Assessment V2,event_name_SUD Brief Assessment V2.1,event_name_SUD Case Management,event_name_SUD Discharge Form,event_name_SUD Inpatient Note,event_name_Safety Plan,event_name_Smoking Status,event_name_Telephone Diagnostic Evaluation,event_name_Telephone EM Service,event_name_Telephone Psychotherapy,event_name_Treatment Plan Signatures,event_name_UA Test,event_name_Urgent Care - Care Coordination,event_name_VA Risk Assessment,event_name_Vocational Rehabilitation,event_name_WHC Daily Routine,event_name_Youth Assessment,event_name_Youth Discharge/Transfer Summary,profile_name_01 Team Butte High 1,profile_name_01 Team Clinton,profile_name_01 Team Hellgate Elem,profile_name_02 Team Butte High 2,profile_name_02 Team CS Porter,profile_name_02 Team Clinton 2,profile_name_02 Team Hellgate Elem 2,profile_name_02 Team Lolo 2,profile_name_03 Team Big Sky 1,profile_name_03 Team Hamilton High School,profile_name_03 Team Hellgate Elem 3,profile_name_03 Team Lolo School Dist 7 #3,profile_name_04 Team Hellgate Elementary 4,profile_name_04 Team Hellgate High,profile_name_04 Team Plains S

In [40]:
X = ml_df.drop(columns=['is_noshow']) # all columns except the target column
y = ml_df['is_noshow'] # target column
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [41]:
from sklearn.linear_model import LogisticRegression

# Create an instance of the LogisticRegression class
logreg = LogisticRegression()

# Fit the model to the training data
logreg.fit(x_train, y_train)


LogisticRegression()

In [51]:
# make predictions on the test data
y_pred = logreg.predict(x_test)

# evaluate the model's performance by calculating the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.4f}%".format(accuracy * 100))
conf_matrix = confusion_matrix(y_test, y_pred)
print('')
print(conf_matrix)

Accuracy: 95.9398%

[[149147      0]
 [  6312      0]]
